In [1]:
import sys
import os
from dotenv import load_dotenv
load_dotenv()
sys.path.insert(0, os.getenv('SRC_PATH'))

import numpy as np
import pandas as pd
import seaborn as sns
import sqlite3
from src.volsurface import GridInterpVolSurface, KernelVolSurface
from src.utils.data_helper import clean_data, VolSurfPointwiseDataset

import torch
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from src.train import Trainer

import json

DB_PATH = os.getenv('DB_PATH')
CSV_PATH = os.getenv('CSV_PATH')
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

pd.set_option('future.no_silent_downcasting', True)

/Users/zhanghq/miniconda3/envs/vae-volsurface/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-18 10:11:15,428	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-04-18 10:11:15,534	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
from src.utils.logger import setup_logger
logger = setup_logger('train')

In [3]:
query = """
SELECT date, symbol, exdate, last_date, cp_flag, strike_price, best_bid, best_offer, volume, open_interest, impl_volatility, delta
FROM opprc
"""
dtype = {
    'symbol': 'string',
    'cp_flag': 'string',
    'strike_price': 'float64',
    'best_bid': 'float64',
    'best_offer': 'float64',
    'volume': 'int64',
    'open_interest': 'int64',
    'impl_volatility': 'float64',
    'delta': 'float64'
}
df_raw = pd.read_sql_query(query, conn, parse_dates=['date', 'exdate', 'last_date'])
df_raw = df_raw.replace('', np.nan) # sqlite returns empty strings for NULL values
df_raw = df_raw.astype(dtype)

df = clean_data(df_raw)

[2025-04-18 10:11:31] [INFO] src.utils.data_helper (49) : Bad data - Filtered 1110238 rows, Retained sample 76.67%
[2025-04-18 10:11:44] [INFO] src.utils.data_helper (62) : Consecutive trading stats completed
[2025-04-18 10:11:45] [INFO] src.utils.data_helper (85) : Consecutive trading - Filtered 2341950 rows, Retained sample 35.81%
[2025-04-18 10:11:45] [INFO] src.utils.data_helper (90) : Moneyness calculation completed


In [4]:
model_name = "vae_pw_improve"
train_model = True
load_model = False
save_model = False
data_dir = CSV_PATH + "/predicted_vol_surfaces.json"  # Path to the volatility surfaces dataset
batch_size = 32
epochs = 10

In [5]:
maturity_grid = np.array([1, 7, 30, 60, 90, 180, 360, 720])
delta_grid = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [6]:
SRC_PATH = os.getenv('SRC_PATH')
os.chdir(SRC_PATH)
trainer = Trainer(model_name)
trainer.create_model()

[2025-04-18 10:11:45] [INFO] src.train (62) : Using device: mps


In [7]:
with open(data_dir, "r") as f:
    data = json.load(f)

In [8]:
mapping_ids = {dt: i for i, dt in enumerate(sorted(map(lambda x: x[:10], data.keys())))}

In [9]:
df['mapping_ids'] = df['date'].dt.strftime('%Y-%m-%d').map(mapping_ids)

In [10]:
vol_surfaces = []
for key in sorted(data.keys()):
    surface = torch.tensor(data[key], dtype=torch.float32)
    vol_surfaces.append(surface.flatten())  # Flatten 2D to 1D

data_tensor = torch.stack(vol_surfaces)
pw_grid_data = torch.tensor(df[['ttm', 'moneyness']].values, dtype=torch.float32)
# !only for test run
pw_grid_data[:, 0] = pw_grid_data[:, 0] / 365.0
pw_vol_data = torch.tensor(df['impl_volatility'].values, dtype=torch.float32)
mapping_ids = torch.tensor(df['mapping_ids'].values).long()

assert data_tensor.shape[0] == max(mapping_ids) + 1

In [11]:
dataset = VolSurfPointwiseDataset(pw_grid_data, pw_vol_data, data_tensor, mapping_ids)
train_loader = DataLoader(
    dataset, 
    batch_size=trainer.batch_size,
    shuffle=True
)

In [12]:
trainer.hypertune(train_loader)

2025-04-18 10:11:50,513	INFO worker.py:1812 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2025-04-18 10:11:51,307	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2025-04-18 10:11:51,309	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
E0000 00:00:1744985511.472178 20011410 tcp_posix.cc:596] recvmsg encountered uncommon error: Message too long


== Status ==
Current time: 2025-04-18 10:11:51 (running for 00:00:00.30)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Logical resource usage: 0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2025-04-18_10-11-49_216658_49254/artifacts/2025-04-18_10-11-51/lambda_2025-04-18_10-11-51/driver_artifacts
Number of trials: 1/1 (1 PENDING)
+--------------------+----------+-------+--------------+--------------+-------------+--------------+-----------------+
| Trial name         | status   | loc   |   batch_size | hidden_dim   |   input_dim |   latent_dim |   learning_rate |
|--------------------+----------+-------+--------------+--------------+-------------+--------------+-----------------|
| lambda_12fb2_00000 | PENDING  |       |           64 | [32, 32]     |          72 |            4 |           0.001 |
+--------------------+----------+-------+--------------+--------------+-------------+--------------+-------------

(pid=49295) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=49295) E0000 00:00:1744985512.408418 20011458 tcp_posix.cc:596] recvmsg encountered uncommon error: Message too long


== Status ==
Current time: 2025-04-18 10:11:56 (running for 00:00:05.32)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Logical resource usage: 2.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2025-04-18_10-11-49_216658_49254/artifacts/2025-04-18_10-11-51/lambda_2025-04-18_10-11-51/driver_artifacts
Number of trials: 1/1 (1 RUNNING)
+--------------------+----------+-----------------+--------------+--------------+-------------+--------------+-----------------+
| Trial name         | status   | loc             |   batch_size | hidden_dim   |   input_dim |   latent_dim |   learning_rate |
|--------------------+----------+-----------------+--------------+--------------+-------------+--------------+-----------------|
| lambda_12fb2_00000 | RUNNING  | 127.0.0.1:49295 |           64 | [32, 32]     |          72 |            4 |           0.001 |
+--------------------+----------+-----------------+--------------+-----

(<lambda> pid=49295) [2025-04-18 10:13:27] [INFO] src.train (181) : Loss: 0.0058


...

(<lambda> pid=49295) [2025-04-18 10:25:40] [INFO] src.train (181) : Loss: 0.0037


== Status ==
Current time: 2025-04-18 10:25:45 (running for 00:13:53.66)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: -4987.453063264489 | Iter 1.000: -7559.7971227020025
Logical resource usage: 2.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2025-04-18_10-11-49_216658_49254/artifacts/2025-04-18_10-11-51/lambda_2025-04-18_10-11-51/driver_artifacts
Number of trials: 1/1 (1 RUNNING)
+--------------------+----------+-----------------+--------------+--------------+-------------+--------------+-----------------+---------+----------------------+
| Trial name         | status   | loc             |   batch_size | hidden_dim   |   input_dim |   latent_dim |   learning_rate |    loss |   training_iteration |
|--------------------+----------+-----------------+--------------+--------------+-------------+--------------+-----------------+---------+----------------------|
| lambda_12fb2_00000 | RUNNING  | 127.0.0.1:49295 |           64 | [32, 

2025-04-18 10:27:11,957	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/zhanghq/ray_results/lambda_2025-04-18_10-11-51' in 0.0034s.
2025-04-18 10:27:11,959	INFO tune.py:1041 -- Total run time: 920.65 seconds (920.50 seconds for the tuning loop).


== Status ==
Current time: 2025-04-18 10:27:11 (running for 00:15:20.50)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: -4987.453063264489 | Iter 1.000: -7559.7971227020025
Logical resource usage: 2.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2025-04-18_10-11-49_216658_49254/artifacts/2025-04-18_10-11-51/lambda_2025-04-18_10-11-51/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)
+--------------------+------------+-----------------+--------------+--------------+-------------+--------------+-----------------+---------+----------------------+
| Trial name         | status     | loc             |   batch_size | hidden_dim   |   input_dim |   latent_dim |   learning_rate |    loss |   training_iteration |
|--------------------+------------+-----------------+--------------+--------------+-------------+--------------+-----------------+---------+----------------------|
| lambda_12fb2_00000 | TERMINATED | 127.0.0.1:49295 |          

(<lambda> pid=49295) [2025-04-18 10:27:11] [INFO] src.train (181) : Loss: 0.0036
(pid=gcs_server) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=gcs_server) I0000 00:00:1744986431.970868 20010625 chttp2_transport.cc:1182] ipv4:127.0.0.1:64772: Got goaway [2] err=UNAVAILABLE:GOAWAY received; Error code: 2; Debug Text: Cancelling all calls {file:"external/com_github_grpc_grpc/src/core/ext/transport/chttp2/transport/chttp2_transport.cc", file_line:1171, created_time:"2025-04-18T10:27:11.970866-04:00", http2_error:2, grpc_status:14}
